In [1]:
from jsons import read_json_to_dict
from mysql_driver import MySQL
import pandas as pd
from sqlalchemy import create_engine

import sys, os

sys.path.append(os.path.dirname(os.getcwd()))

import utils.folder_tb as fo
import utils.mining_data_tb as md

In [2]:
# Read the json
read_json = read_json_to_dict("sql_server_settings.json")

IP_DNS = read_json["IP_DNS"]
USER = read_json["USER"]
PASSWORD = read_json["PASSWORD"]
DB_NAME = read_json["DB_NAME"]
PORT = read_json["PORT"]

# Daily Intakes data insertion

# Connect to MySQL

mysql_db = MySQL(IP_DNS, USER, PASSWORD, DB_NAME, PORT)
mysql_db.connect()

# Create table

create_table_sql = """CREATE TABLE dailyintakes(
    ID INT(20) NOT NULL AUTO_INCREMENT,
    GENDER VARCHAR(20) NOT NULL,
    AGE INT(2) NOT NULL,
    URL VARCHAR(200),
    PRIMARY KEY (ID))"""

mysql_db.execute_interactive_sql(create_table_sql)

select_sql = """SELECT * FROM dailyintakes"""

mysql_db.execute_get_sql(select_sql)

for ind, row in daily_intake_data.iterrows():
    to_insert = list(row)
    
    to_insert_sql = mysql_db.generate_insert_into_dailyintakes_sql(to_insert)
    mysql_db.execute_interactive_sql(to_insert_sql)

select_sql = """SELECT * FROM dailyintakes"""

mysql_db.execute_get_sql(select_sql)

# More tests

mysql_db.close()

select_sql = """SELECT * FROM dailyintakes"""

mysql_db.execute_get_sql(select_sql)

# Resources data insertion

In [3]:
path2 = fo.path_to_folder(2, "data") + "CLEANED_Resources.csv"
resources_df = pd.read_csv(path2)

resources_df = resources_df.applymap(md.to_string)

column_names = list(resources_df.columns)
column_names.pop(0)
column_names.insert(0, "Food")

resources_df.columns = column_names
resources_df.columns

Index(['Food', 'Total_emissions', 'Land use per 1000kcal', 'Land use per kg',
       'Land use per 100g protein', 'Freswater withdrawls per 1000kcal',
       'Freswater withdrawls per kg', 'Freswater withdrawls per 100g protein',
       'Origin'],
      dtype='object')

In [4]:
resources_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 9 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Food                                   45 non-null     object
 1   Total_emissions                        45 non-null     object
 2   Land use per 1000kcal                  45 non-null     object
 3   Land use per kg                        45 non-null     object
 4   Land use per 100g protein              45 non-null     object
 5   Freswater withdrawls per 1000kcal      45 non-null     object
 6   Freswater withdrawls per kg            45 non-null     object
 7   Freswater withdrawls per 100g protein  45 non-null     object
 8   Origin                                 45 non-null     object
dtypes: object(9)
memory usage: 3.3+ KB


In [5]:
# Connect to MySQL

mysql_db = MySQL(IP_DNS, USER, PASSWORD, DB_NAME, PORT)
mysql_db.connect()

Connected to MySQL server [45395203b_ds_april_2021_db]


In [9]:
# Create table

create_table_sql = """CREATE TABLE resources(
    ID INT(20) AUTO_INCREMENT,
    FOOD VARCHAR(300),
    TOTAL_EMISSIONS INT,
    LAND_USE_PER_1000KCAL INT,
    LAND_USE_PER_KG INT,
    LAND_USE_PER_100G_PROTEIN INT,
    FRESHWATER_WITHDRAWLS_PER_1000KCAL INT,
    FRESHWATER_WITHDRAWLS_PER_KG INT,
    FRESHWATER_WITHDRAWLS_PER_100G_PROTEIN INT,
    ORIGIN VARCHAR(50),
    PRIMARY KEY (ID))"""

mysql_db.execute_interactive_sql(create_table_sql)

Executed 

CREATE TABLE resources(
    ID INT(20) AUTO_INCREMENT,
    FOOD VARCHAR(300),
    TOTAL_EMISSIONS INT,
    LAND_USE_PER_1000KCAL INT,
    LAND_USE_PER_KG INT,
    LAND_USE_PER_100G_PROTEIN INT,
    FRESHWATER_WITHDRAWLS_PER_1000KCAL INT,
    FRESHWATER_WITHDRAWLS_PER_KG INT,
    FRESHWATER_WITHDRAWLS_PER_100G_PROTEIN INT,
    ORIGIN VARCHAR(50),
    PRIMARY KEY (ID))

 succesfully


1

In [18]:
resources_df.head()

,Food,Total_emissions,Land use per 1000kcal,Land use per kg,Land use per 100g protein,Freswater withdrawls per 1000kcal,Freswater withdrawls per kg,Freswater withdrawls per 100g protein,Origin
0,Oatmeal,1.6,2.897445673,7.6,5.846153846,183.9115517,482.4,371.0769231,plant based
1,Rice,4.0,0.759631036,2.8,3.943661972,609.9837222,2248.4,3166.760563,plant based
2,Potatoes,0.3,1.202185792,0.88,5.176470588,80.73770492,59.1,347.6470588,plant based
3,Cassava,0.9,1.858316222,1.81,20.11111111,nan,0.0,nan,plant based
4,Cane Sugar,2.6,0.581196581,2.04,nan,176.6666667,620.1,nan,plant based


In [19]:
db_connection_str = mysql_db.SQL_ALCHEMY
db_connection = create_engine(db_connection_str)

resources_df.to_sql('resources', con = db_connection, index = False)

for ind, row in resources_df.iterrows():
    to_insert = list(row)
    #print(to_insert)
    to_insert_sql = mysql_db.generate_insert_into_resources_sql(to_insert)
    #print(to_insert_sql)
    mysql_db.execute_interactive_sql(to_insert_sql)

# Nutritional values data insertion

In [20]:
path2 = fo.path_to_folder(2, "data") + "CLEANED_Nutritional_values_stats.csv"
nutritional_values_stats = pd.read_csv(path2)

nutritional_values_stats = nutritional_values_stats.applymap(md.to_string)
nutritional_values_stats.head()

,Category 2,Protein (g),Water (g),"Fiber, total dietary (g)","Vitamin A, RAE (mcg_RAE)",Thiamin (mg),Riboflavin (mg),Niacin (mg),Vitamin B-6 (mg),Vitamin B-12 (mcg),...,Sodium (mg),Zinc (mg),Energy (kcal),Total Fat (g),"Fatty acids, total saturated (g)","Fatty acids, total monounsaturated (g)","Fatty acids, total polyunsaturated (g)",Cholesterol (mg),Vitamin D (D2 + D3) (mcg),Carbohydrate (g)
0,_others,6.106123766816131,59.87791210762336,1.8640179372197367,73.56125560538116,0.1741895964125551,0.1783259192825109,2.321376322869969,0.1787167713004477,0.4419156950672639,...,309.1424215246637,1.090423318385658,191.2938116591928,7.90098654708516,2.5168071748878926,2.7165643049327337,2.0021287892376765,18.782959641255605,0.3023497757847525,24.36157488789236
1,beans,8.882972972972972,60.673783783783776,7.198648648648651,2.45945945945946,0.1479459459459459,0.0723648648648648,0.6207837837837838,0.1583783783783783,0.0197297297297297,...,291.72972972972974,1.1345945945945952,173.25675675675674,5.713783783783785,0.9314864864864866,2.1094189189189185,2.3210000000000006,1.4324324324324325,0.0081081081081081,22.774729729729724
2,cheese,20.260958904109582,51.2809589041096,0.1,167.87671232876713,0.0383972602739726,0.3261780821917809,0.257,0.0918082191780822,1.0828767123287673,...,738.2054794520548,2.523972602739728,275.8493150684931,19.127534246575337,11.244191780821918,5.113616438356164,0.9789589041095894,61.31506849315068,0.8287671232876712,5.557534246575344
3,chicken,23.178750000000008,60.54558035714287,0.1593749999999999,15.071428571428571,0.0890133928571428,0.2037410714285714,6.596642857142853,0.4022008928571429,0.4730803571428573,...,453.13392857142856,1.7108035714285716,214.375,11.682946428571432,2.93689732142857,4.692308035714286,2.7313571428571417,101.84375,0.1334821428571428,3.249776785714285
4,fish,21.006612903225808,63.88760368663595,0.2875576036866351,49.99769585253456,0.123889400921659,0.1507465437788017,4.530237327188939,0.2868778801843319,3.5892626728110626,...,478.70046082949307,2.1891705069124434,177.1336405529954,7.109631336405526,1.6488755760368663,2.5525990783410126,2.0145115207373268,85.36175115207374,4.903686635944695,6.001958525345614


In [22]:
db_connection_str_2 = mysql_db.SQL_ALCHEMY
db_connection_2 = create_engine(db_connection_str_2)

nutritional_values_stats.to_sql('nutritional_values_stats', con = db_connection_2, index = False)

### Nutritional values (the largest dataset)

In [23]:
path3 = fo.path_to_folder(2, "data") + "CLEANED_Nutritional_values.csv"
nutritional_values = pd.read_csv(path3)

nutritional_values = nutritional_values.applymap(md.to_string)
nutritional_values.head()

,Food name,Category number,Category name,Protein (g),Water (g),"Fiber, total dietary (g)","Vitamin A, RAE (mcg_RAE)",Thiamin (mg),Riboflavin (mg),Niacin (mg),...,Energy (kcal),Total Fat (g),"Fatty acids, total saturated (g)","Fatty acids, total monounsaturated (g)","Fatty acids, total polyunsaturated (g)",Cholesterol (mg),Vitamin D (D2 + D3) (mcg),Carbohydrate (g),Category 2,Category 3
0,"Milk, human",9602,Human milk,1.03,87.5,0.0,61,0.014,0.036,0.177,...,70,4.38,2.009,1.658,0.497,14,0.1,6.89,milks,animal
1,"Milk, NFS",1004,"Milk, reduced fat",3.34,89.04,0.0,59,0.057,0.137,0.11,...,51,1.99,1.164,0.426,0.065,8,1.1,4.87,milks,animal
2,"Milk, whole",1002,"Milk, whole",3.28,88.1,0.0,32,0.056,0.138,0.105,...,60,3.2,1.86,0.688,0.108,12,1.1,4.67,milks,animal
3,"Milk, low sodium, whole",1002,"Milk, whole",3.1,88.2,0.0,29,0.02,0.105,0.043,...,61,3.46,2.154,0.999,0.128,14,1.3,4.46,milks,animal
4,"Milk, calcium fortified, whole",1002,"Milk, whole",3.28,88.1,0.0,32,0.056,0.138,0.105,...,60,3.2,1.86,0.688,0.108,12,1.1,4.67,milks,animal


In [25]:
db_connection_str_3 = mysql_db.SQL_ALCHEMY
db_connection_3 = create_engine(db_connection_str_3)

nutritional_values.to_sql('nutritional_values', con = db_connection_3, index = False)

print("FINISHED")

FINISHED


In [26]:
#mysql_db.close()

db = mysql_db.connect()

select_sql = """SELECT * FROM nutritional_values"""

test = pd.read_sql("select * from nutritional_values", con = db)
test.head()

Connected to MySQL server [45395203b_ds_april_2021_db]


,Food name,Category number,Category name,Protein (g),Water (g),"Fiber, total dietary (g)","Vitamin A, RAE (mcg_RAE)",Thiamin (mg),Riboflavin (mg),Niacin (mg),...,Energy (kcal),Total Fat (g),"Fatty acids, total saturated (g)","Fatty acids, total monounsaturated (g)","Fatty acids, total polyunsaturated (g)",Cholesterol (mg),Vitamin D (D2 + D3) (mcg),Carbohydrate (g),Category 2,Category 3
0,"Milk, human",9602,Human milk,1.03,87.5,0.0,61,0.014,0.036,0.177,...,70,4.38,2.009,1.658,0.497,14,0.1,6.89,milks,animal
1,"Milk, NFS",1004,"Milk, reduced fat",3.34,89.04,0.0,59,0.057,0.137,0.11,...,51,1.99,1.164,0.426,0.065,8,1.1,4.87,milks,animal
2,"Milk, whole",1002,"Milk, whole",3.28,88.1,0.0,32,0.056,0.138,0.105,...,60,3.2,1.86,0.688,0.108,12,1.1,4.67,milks,animal
3,"Milk, low sodium, whole",1002,"Milk, whole",3.1,88.2,0.0,29,0.02,0.105,0.043,...,61,3.46,2.154,0.999,0.128,14,1.3,4.46,milks,animal
4,"Milk, calcium fortified, whole",1002,"Milk, whole",3.28,88.1,0.0,32,0.056,0.138,0.105,...,60,3.2,1.86,0.688,0.108,12,1.1,4.67,milks,animal


In [29]:
mysql_db.close()

Close connection with MySQL server [45395203b_ds_april_2021_db]
